In [1]:
#!pip install rioxarray
import dask.array as da
import pystac_client
from pystac_client import Client
import leafmap
from datetime import datetime
import dask
import planetary_computer as pc
import geogif
import numpy as np
import xarray as xr
import rioxarray
import geopandas as gpd
import matplotlib.pyplot as plt
import geojson
import json
from geogif import dgif, gif
import pandas as pd
import shapely
from shapely.geometry import mapping
from shapely.geometry import box
import folium
from pystac import ItemCollection
from pyproj import CRS
from branca.colormap import LinearColormap

In [2]:
pov = gpd.read_file("InputData/woo_poverty_2020.shp")
ed = gpd.read_file("InputData/woo_education_2020.shp")
#ed.head()

In [3]:
print(len(pov))

pov.head()

46


,OBJECTID,GEOID,ALAND,AWATER,NAME,State,County,pop,pop_ME,popPov,popPov_ME,Shape_Leng,Shape_Area,PovPer,geometry
0,1,25027730100,5625835.0,9057.0,Census Tract 7301,Massachusetts,Worcester County,6973,913.0,415,230.0,16930.397385,1.031319e+07,5.951527,"POLYGON ((269820.112 4689563.097, 269801.016 4..."
1,2,25027730200,4059335.0,10906.0,Census Tract 7302,Massachusetts,Worcester County,6865,789.0,784,454.0,15188.449082,7.449705e+06,11.420248,"POLYGON ((270706.789 4690826.668, 270758.335 4..."
2,3,25027730300,2005168.0,0.0,Census Tract 7303,Massachusetts,Worcester County,5325,723.0,654,424.0,10029.277134,3.668713e+06,12.281690,"POLYGON ((270061.440 4686842.810, 270053.851 4..."
3,4,25027730401,2691314.0,28341.0,Census Tract 7304.01,Massachusetts,Worcester County,5664,782.0,1153,430.0,12996.255886,4.974090e+06,20.356638,"POLYGON ((271360.760 4687153.050, 271418.047 4..."
4,5,25027730402,720557.0,0.0,Census Tract 7304.02,Massachusetts,Worcester County,2037,246.0,350,219.0,4870.844466,1.317466e+06,17.182131,"POLYGON ((270463.517 4685784.119, 270439.303 4..."


In [4]:
tracts= gpd.read_file("InputData/woo_tracts_32619.shp")
#tracts.head()

In [5]:
print((pov.columns).tolist())

['OBJECTID', 'GEOID', 'ALAND', 'AWATER', 'NAME', 'State', 'County', 'pop', 'pop_ME', 'popPov', 'popPov_ME', 'Shape_Leng', 'Shape_Area', 'PovPer', 'geometry']


In [6]:
# Get rid of More columns
columns_to_delete = ['ALAND', 'AWATER', 'NAME', 'State', 'County','Shape_Leng', 'Shape_Area','pop_ME', 'popPov_ME']
pov = pov.drop(columns=columns_to_delete)
pov.head()

,OBJECTID,GEOID,pop,popPov,PovPer,geometry
0,1,25027730100,6973,415,5.951527,"POLYGON ((269820.112 4689563.097, 269801.016 4..."
1,2,25027730200,6865,784,11.420248,"POLYGON ((270706.789 4690826.668, 270758.335 4..."
2,3,25027730300,5325,654,12.281690,"POLYGON ((270061.440 4686842.810, 270053.851 4..."
3,4,25027730401,5664,1153,20.356638,"POLYGON ((271360.760 4687153.050, 271418.047 4..."
4,5,25027730402,2037,350,17.182131,"POLYGON ((270463.517 4685784.119, 270439.303 4..."


In [7]:
print((ed.columns).tolist())

['OBJECTID', 'GEOID', 'ALAND', 'AWATER', 'NAME', 'State', 'County', 'B15002_001', 'B15002_002', 'popBach', 'popBach_ma', 'perBach', 'perBach_ma', 'Shape_Leng', 'Shape_Area', 'geometry']


In [8]:
# Get rid of More columns
columns_to_delete = ['ALAND', 'AWATER', 'NAME', 'State', 'County','Shape_Leng', 'Shape_Area', 'B15002_002','popBach_ma','perBach_ma']
ed = ed.drop(columns=columns_to_delete)
# Rename esoteric columns
ed.rename(columns={'B15002_001': 'pop'}, inplace=True)
ed.head()

,OBJECTID,GEOID,pop,popBach,perBach,geometry
0,1,25027730100,4769,2136,44.8,"POLYGON ((269820.112 4689563.097, 269801.016 4..."
1,2,25027730200,4821,1951,40.5,"POLYGON ((270706.789 4690826.668, 270758.335 4..."
2,3,25027730300,3998,1374,34.4,"POLYGON ((270061.440 4686842.810, 270053.851 4..."
3,4,25027730401,3908,1335,34.2,"POLYGON ((271360.760 4687153.050, 271418.047 4..."
4,5,25027730402,1296,253,19.5,"POLYGON ((270463.517 4685784.119, 270439.303 4..."


In [9]:
# create a nice 'Tract' column
ed['GEOID'] = ed['GEOID'].astype(str)

# Extract the last 6 digits of each 'GEOID' and assign it to the 'Tract' column
ed['Tract'] = ed['GEOID'].str[-6:]
ed = ed.drop(columns='GEOID')
ed=ed.sort_values(by='Tract')
ed.head()

,OBJECTID,pop,popBach,perBach,geometry,Tract
0,1,4769,2136,44.8,"POLYGON ((269820.112 4689563.097, 269801.016 4...",730100
1,2,4821,1951,40.5,"POLYGON ((270706.789 4690826.668, 270758.335 4...",730200
2,3,3998,1374,34.4,"POLYGON ((270061.440 4686842.810, 270053.851 4...",730300
3,4,3908,1335,34.2,"POLYGON ((271360.760 4687153.050, 271418.047 4...",730401
4,5,1296,253,19.5,"POLYGON ((270463.517 4685784.119, 270439.303 4...",730402


In [10]:
# do the same thing for pov
pov['GEOID'] = pov['GEOID'].astype(str)

# Extract the last 6 digits of each 'GEOID' and assign it to the 'Tract' column
pov['Tract'] = pov['GEOID'].str[-6:]
pov = pov.drop(columns='GEOID')
pov=pov.sort_values(by='Tract')
pov.rename(columns={'PovPer': 'perPov'}, inplace=True)

pov = pov.reset_index(drop=True)

pov.head()

,OBJECTID,pop,popPov,perPov,geometry,Tract
0,1,6973,415,5.951527,"POLYGON ((269820.112 4689563.097, 269801.016 4...",730100
1,2,6865,784,11.420248,"POLYGON ((270706.789 4690826.668, 270758.335 4...",730200
2,3,5325,654,12.281690,"POLYGON ((270061.440 4686842.810, 270053.851 4...",730300
3,4,5664,1153,20.356638,"POLYGON ((271360.760 4687153.050, 271418.047 4...",730401
4,5,2037,350,17.182131,"POLYGON ((270463.517 4685784.119, 270439.303 4...",730402


In [11]:
# ed4326 = ed.to_crs(epsg=4326)
# pov4326 = pov.to_crs(epsg=4326)
# tracts4326 = tracts.to_crs(epsg=4326)

In [12]:
# # On this map, if you turn off the 2010 layer, you can get the tract names by hovering the mouse
# # We can see that 731600 split into 731601 and 731602, and 731800 into 731801 and 731802
# # That is annoying!
# import folium

# # Create a Folium map
# m = folium.Map(location=[42.2626, -71.8023], zoom_start=11.5)
# colormap = LinearColormap(colors=['green', 'white'], vmin=17.7, vmax=88.7)


# # Create a feature group for ed4326
# ed_group = folium.FeatureGroup(name='Woo ed')

# # Iterate over the tracts in woo_2010_4326 and add them to the map without applying any fill color
# for idx, row in ed4326.iterrows():
#     # Add GeoJSON feature to the map without any fill color
#     folium.GeoJson(row.geometry.__geo_interface__,
#                    style_function=lambda x: {'fillColor': 'transparent'}),
    


    
#     tooltip=f"Census Tract: {tract_name}").add_to(ed_group)



# # Add woo2010_group to the map
# ed_group.add_to(m)


# # Create a feature group for ed4326
# pov_group = folium.FeatureGroup(name='Woo pov')

# # Iterate over the tracts in pov4326 and add them to the map without applying any fill color
# for idx, row in pov4326.iterrows():
#     # Add GeoJSON feature to the map without any fill color
#     folium.GeoJson(row.geometry.__geo_interface__, 
#                    style_function=lambda x: {'fillColor': 'transparent'}).add_to(pov_group)

# # Add woo2010_group to the map
# pov_group.add_to(m)


# # Create a feature group for ed4326
# tracts_group = folium.FeatureGroup(name='2010 tracts')

# # Iterate over the tracts in tracts4326 and add them to the map without applying any fill color
# for idx, row in tracts4326.iterrows():
#     # Add GeoJSON feature to the map without any fill color
#     folium.GeoJson(row.geometry.__geo_interface__, 
#                    style_function=lambda x: {'fillColor': 'transparent'}).add_to(tracts_group)

# # Add woo2010_group to the map
# tracts_group.add_to(m)


# # Add layer control to toggle layers on/off
# folium.LayerControl().add_to(m)

# # Display the map
# m


In [13]:
print(len(pov['Tract'].unique()))
print(ed)

46
    OBJECTID   pop  popBach  perBach  \
0          1  4769     2136     44.8   
1          2  4821     1951     40.5   
2          3  3998     1374     34.4   
3          4  3908     1335     34.2   
4          5  1296      253     19.5   
5          6  2400      746     31.1   
6          7  5932     3170     53.4   
7          8  5330     2676     50.2   
8          9  3065     1987     64.8   
9         10  1596      895     56.1   
10        11  2898     1405     48.5   
11        12  1963      846     43.1   
12        13  2279      411     18.0   
13        14  4768     1353     28.4   
14        15  2602      648     24.9   
15        16  2616     1311     50.1   
16        17     0        0      0.0   
17        18  3111      678     21.8   
18        19  1503      270     18.0   
19        20  3008      394     13.1   
20        21  3271      474     14.5   
21        22  3541      644     18.2   
42        43  2776     1114     40.1   
43        44   353      183     51.8 

In [15]:
print(len(tracts))
print(len(ed))
print(len(pov))

44
46
46


In [14]:
# Looks like Tract 731600, right in the middle of Worcester, was split along Highland Street into 731601 and 731602, and the same thing with 731800 to 731801 and 731802
# In order to compare 2020 with 2010 we will have to merge these
# Very tedious

# Filter rows for Tract 731601 and Tract 731602
ed_731601 = ed[ed['Tract'] == '731601']
ed_731602 = ed[ed['Tract'] == '731602']

# Extract geometry
t1 = ed_731601['geometry']
t2 = ed_731602['geometry']
# Concatenate
combined_rows = pd.concat([t1, t2], ignore_index=True)
# Unify
combined = combined_rows.unary_union
ed_731600 = gpd.GeoDataFrame(geometry=[combined])
ed_731600= ed_731600.set_crs("EPSG:32619")
print(type(ed_731600))
ed_731600

<class 'geopandas.geodataframe.GeoDataFrame'>


,geometry
0,"POLYGON ((268449.787 4682890.791, 268352.929 4..."


In [15]:
# Create a dictionary to store the sums of each column
sums = {}

# List of columns to sum
columns_to_sum = ['pop', 'popBach']

# Iterate over each column and calculate the sum
for column in columns_to_sum:
    sums[column] = ed_731601[column].sum() + ed_731602[column].sum()

# Add the sums as new columns to combined_gdf
for column, value in sums.items():
    ed_731600[column] = value

# Print the updated combined_gdf
ed_731600

,geometry,pop,popBach
0,"POLYGON ((268449.787 4682890.791, 268352.929 4...",3129,1297


In [16]:
total_pop = ed_731600['pop']
ed_731600['perBach'] = ((ed_731600['popBach'] / total_pop) * 100).round(2)
ed_731600['Tract'] = '731600'
ed_731600

,geometry,pop,popBach,perBach,Tract
0,"POLYGON ((268449.787 4682890.791, 268352.929 4...",3129,1297,41.45,731600


In [17]:
# Filter rows for Tract 731801 and Tract 731802
ed_731801 = ed[ed['Tract'] == '731801']
ed_731802 = ed[ed['Tract'] == '731802']

# Extract geometry
t1 = ed_731801['geometry']
t2 = ed_731802['geometry']
# Concatenate
combined_rows = pd.concat([t1, t2], ignore_index=True)
# Unify
combined = combined_rows.unary_union
ed_731800 = gpd.GeoDataFrame(geometry=[combined])
ed_731800= ed_731800.set_crs("EPSG:32619")
# Create a dictionary to store the sums of each column
sums = {}
# List of columns to sum
columns_to_sum = ['pop', 'popBach']
# Iterate over each column and calculate the sum
for column in columns_to_sum:
    sums[column] = ed_731801[column].sum() + ed_731801[column].sum()
# Add the sums as new columns to combined_gdf
for column, value in sums.items():
    ed_731800[column] = value
# Assign new values
total_pop = ed_731800['pop']
ed_731800['perBach'] = ((ed_731800['popBach'] / total_pop) * 100).round(2)
ed_731800['Tract'] = '731800'
ed_731800

,geometry,pop,popBach,perBach,Tract
0,"POLYGON ((271536.273 4683700.890, 271491.730 4...",4474,1194,26.69,731800


In [18]:
# First drop the old subdivided tracts
ed = ed.drop(ed_731801.index)
ed = ed.drop(ed_731802.index)
ed = ed.drop(ed_731601.index)
ed = ed.drop(ed_731602.index)

ed = ed.sort_values(by='Tract')

# Reset the index
ed = ed.reset_index(drop=True)


ed1 = pd.concat([ed, ed_731600, ed_731800], ignore_index=True)
ed = ed.sort_values(by='Tract')

# Reset the index
ed = ed.reset_index(drop=True)
len(ed1)
ed1

,OBJECTID,pop,popBach,perBach,geometry,Tract
0,1.0,4769,2136,44.80,"POLYGON ((269820.112 4689563.097, 269801.016 4...",730100
1,2.0,4821,1951,40.50,"POLYGON ((270706.789 4690826.668, 270758.335 4...",730200
2,3.0,3998,1374,34.40,"POLYGON ((270061.440 4686842.810, 270053.851 4...",730300
3,4.0,3908,1335,34.20,"POLYGON ((271360.760 4687153.050, 271418.047 4...",730401
4,5.0,1296,253,19.50,"POLYGON ((270463.517 4685784.119, 270439.303 4...",730402
5,6.0,2400,746,31.10,"POLYGON ((269091.659 4687128.029, 269093.735 4...",730500
6,7.0,5932,3170,53.40,"POLYGON ((267835.969 4687451.977, 267849.975 4...",730600
7,8.0,5330,2676,50.20,"POLYGON ((266305.992 4686116.730, 266297.544 4...",730700
8,9.0,3065,1987,64.80,"POLYGON ((268341.280 4684424.153, 268263.534 4...",730801
9,10.0,1596,895,56.10,"POLYGON ((267980.794 4683955.814, 267917.925 4...",730802


In [19]:
print(pov)

    OBJECTID   pop  popPov     perPov  \
0          1  6973     415   5.951527   
1          2  6865     784  11.420248   
2          3  5325     654  12.281690   
3          4  5664    1153  20.356638   
4          5  2037     350  17.182131   
5          6  3785     670  17.701453   
6          7  7783     323   4.150071   
7          8  7561    1055  13.953181   
8          9  4082     211   5.169035   
9         10  2250     179   7.955556   
10        11  3782     427  11.290323   
11        12  2690     557  20.706320   
12        13  2902     120   4.135079   
13        14  6504    1263  19.418819   
14        15  3612     430  11.904762   
15        16  3487     227   6.509894   
16        17     0       0   0.000000   
17        18  5398    2121  39.292330   
18        19  2484     618  24.879227   
19        20  5035    1821  36.166832   
20        21  4566    1688  36.968901   
21        22  5061    1415  27.958901   
22        43  4156    1494  35.948027   
23        44  10

In [20]:
# Filter rows for Tract 731601 and Tract 731602
pov_731801 = pov[pov['Tract'] == '731801']
pov_731802 = pov[pov['Tract'] == '731802']

# Extract geometry
t1 = pov_731801['geometry']
t2 = pov_731802['geometry']
# Concatenate
combined_rows = pd.concat([t1, t2], ignore_index=True)
# Unify
combined = combined_rows.unary_union
pov_731800 = gpd.GeoDataFrame(geometry=[combined])
pov_731800= pov_731800.set_crs("EPSG:32619")
# Create a dictionary to store the sums of each column
sums = {}
# List of columns to sum
columns_to_sum = ['pop', 'popPov']
# Iterate over each column and calculate the sum
for column in columns_to_sum:
    sums[column] = pov_731801[column].sum() + pov_731801[column].sum()
# Add the sums as new columns to combined_gdf
for column, value in sums.items():
    pov_731800[column] = value
# Assign new values
total_pop = pov_731800['pop']
pov_731800['perPov'] = ((pov_731800['popPov'] / total_pop) * 100).round(2)
pov_731800['Tract'] = '731800'
pov_731800

,geometry,pop,popPov,perPov,Tract
0,"POLYGON ((271536.273 4683700.890, 271491.730 4...",6010,2084,34.68,731800


In [21]:
# Filter rows for Tract 731601 and Tract 731602
pov_731601 = pov[pov['Tract'] == '731601']
pov_731602 = pov[pov['Tract'] == '731602']

# Extract geometry
t1 = pov_731601['geometry']
t2 = pov_731602['geometry']
# Concatenate
combined_rows = pd.concat([t1, t2], ignore_index=True)
# Unify
combined = combined_rows.unary_union
pov_731600 = gpd.GeoDataFrame(geometry=[combined])
pov_731600= pov_731600.set_crs("EPSG:32619")
# Create a dictionary to store the sums of each column
sums = {}
# List of columns to sum
columns_to_sum = ['pop', 'popPov']
# Iterate over each column and calculate the sum
for column in columns_to_sum:
    sums[column] = pov_731601[column].sum() + pov_731601[column].sum()
# Add the sums as new columns to combined_gdf
for column, value in sums.items():
    pov_731600[column] = value
# Assign new values
total_pop = pov_731600['pop']
pov_731600['perPov'] = ((pov_731600['popPov'] / total_pop) * 100).round(2)
pov_731600['Tract'] = '731600'
pov_731600

,geometry,pop,popPov,perPov,Tract
0,"POLYGON ((268449.787 4682890.791, 268352.929 4...",8312,2988,35.95,731600


In [24]:
len(pov)

46

In [22]:
# Extract indices of rows where Tract is '731601' and '731602'
indices_to_drop = pov[pov['Tract'].isin(['731601', '731602','731801','731802'])].index

# Drop the rows using the extracted indices
pov = pov.drop(indices_to_drop)


In [23]:
pov1 = pd.concat([pov, pov_731600, pov_731800], ignore_index=True)
len(pov1)

44

In [24]:
pov1

,OBJECTID,pop,popPov,perPov,geometry,Tract
0,1.0,6973,415,5.951527,"POLYGON ((269820.112 4689563.097, 269801.016 4...",730100
1,2.0,6865,784,11.420248,"POLYGON ((270706.789 4690826.668, 270758.335 4...",730200
2,3.0,5325,654,12.281690,"POLYGON ((270061.440 4686842.810, 270053.851 4...",730300
3,4.0,5664,1153,20.356638,"POLYGON ((271360.760 4687153.050, 271418.047 4...",730401
4,5.0,2037,350,17.182131,"POLYGON ((270463.517 4685784.119, 270439.303 4...",730402
5,6.0,3785,670,17.701453,"POLYGON ((269091.659 4687128.029, 269093.735 4...",730500
6,7.0,7783,323,4.150071,"POLYGON ((267835.969 4687451.977, 267849.975 4...",730600
7,8.0,7561,1055,13.953181,"POLYGON ((266305.992 4686116.730, 266297.544 4...",730700
8,9.0,4082,211,5.169035,"POLYGON ((268341.280 4684424.153, 268263.534 4...",730801
9,10.0,2250,179,7.955556,"POLYGON ((267980.794 4683955.814, 267917.925 4...",730802


In [29]:
pov4326 = pov1.to_crs('4326')

In [31]:
pov4326.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [30]:
# On this map, if you turn off the 2010 layer, you can get the tract names by hovering the mouse
# We can see that 731600 split into 731601 and 731602, and 731800 into 731801 and 731802
# That is annoying!
import folium

# Create a Folium map
m = folium.Map(location=[42.2626, -71.8023], zoom_start=11.5)

# Iterate over the tracts in woo_2010_4326 and add them to the map without applying any fill color
for idx, row in pov1.iterrows():
    # Add GeoJSON feature to the map without any fill color
    folium.GeoJson(row.geometry.__geo_interface__, 
                   style_function=lambda x: {'fillColor': 'transparent'}).add_to(m)


# Display the map
m


In [25]:
pov1.to_file("woo_pov.shp")

In [25]:
# First drop the old subdivided tracts
pov = pov.drop(pov[pov['Tract'] == '731601'])
pov = pov.drop(pov[pov['Tract'] == '731602'])
pov = pov.drop(pov[pov['Tract'] == '731801'])
pov = pov.drop(pov[pov['Tract'] == '731802'])

pov1 = pd.concat([pov, pov_731600, pov_731800], ignore_index=True)
len(pov1)

KeyError: "['OBJECTID', 'pop', 'popPov', 'PovPer', 'geometry', 'Tract'] not found in axis"

In [27]:
# pov1.to_file("woo_pov.shp")
# ed1.to_file("woo_ed.shp")
